In [2]:
# load gold_text, gold_rationales
# load transcript
import json

filepath = "/mnt/hdd4/jinmyeong/hts_data/[FINAL-v7].json"

with open(filepath, 'r') as fp:
    data = json.load(fp)

In [4]:
data[0]

{'audio_id': '1118579093153382400_twitter',
 'sentence': ['mmh', 'alsterwasser', 'grapefruit'],
 'intervals': {'intervals [1]': {'xmin': 0.0, 'xmax': 0.25, 'text': 'mmh'},
  'intervals [2]': {'xmin': 0.25, 'xmax': 1.05, 'text': 'alsterwasser'},
  'intervals [3]': {'xmin': 1.05, 'xmax': 1.58, 'text': 'grapefruit'}},
 'rationale': [],
 'label': 'normal'}

In [3]:
audio_rationales_data = {}
for audio_datapoint in data:
    audio_id = audio_datapoint['audio_id']
    audio_rationales_data[audio_id] = audio_datapoint
file_path = '/home/jinmyeong/code/hts/classifier/Data/audio_rationales_data.json'
with open(file_path, "w") as f:
    json.dump(audio_rationales_data, f)

In [5]:
with open("/home/jinmyeong/code/hts/classifier/Data/gold_hatemm_audioHateXplain.json", "r") as f:
    audio_rationales_data = json.load(f)


In [4]:
import jiwer

with open('/home/jinmyeong/code/hts/classifier/Data/new_post_id_divisions.json', 'r') as fp:
    post_id_division = json.load(fp)

with open('/home/jinmyeong/code/hts/ASR/Data/ASR/whisperX_ASR_transcription_v4-HateSpeech-all-multispeaker.json', 'r') as fp:
    transcript_dict = json.load(fp)

print(transcript_dict["1178765655790755845_twitter"])

[{'start': 0.149, 'end': 3.339, 'text': ' User, why would anyone watch wigger garbage like this show in the beginning?', 'words': [{'word': 'User,', 'start': 0.149, 'end': 0.41, 'score': 0.915}, {'word': 'why', 'start': 0.49, 'end': 0.631, 'score': 0.998}, {'word': 'would', 'start': 0.651, 'end': 0.811, 'score': 0.847}, {'word': 'anyone', 'start': 0.872, 'end': 1.172, 'score': 0.944}, {'word': 'watch', 'start': 1.213, 'end': 1.433, 'score': 0.873}, {'word': 'wigger', 'start': 1.493, 'end': 1.754, 'score': 0.886}, {'word': 'garbage', 'start': 1.794, 'end': 2.155, 'score': 0.923}, {'word': 'like', 'start': 2.215, 'end': 2.376, 'score': 0.834}, {'word': 'this', 'start': 2.416, 'end': 2.576, 'score': 0.912}, {'word': 'show', 'start': 2.617, 'end': 2.797, 'score': 0.911}, {'word': 'in', 'start': 2.837, 'end': 2.897, 'score': 0.748}, {'word': 'the', 'start': 2.917, 'end': 2.998, 'score': 0.833}, {'word': 'beginning?', 'start': 3.018, 'end': 3.339, 'score': 0.935}]}]


In [6]:
import jiwer

with open('/home/jinmyeong/code/hts/classifier/Data/hatemm_post_id_divisions.json', 'r') as fp:
    post_id_division = json.load(fp)

with open('/home/jinmyeong/code/hts/ASR/Data/ASR/whisperX_ASR_transcription_hatemm_hate.json', 'r') as fp:
    transcript_dict = json.load(fp)

In [7]:


id_list = post_id_division['train'] + post_id_division['val'] + post_id_division['test']

transcript_rationales_dict = {}

for test_id in id_list:
    if audio_rationales_data.get(test_id):
        audio_datapoint = audio_rationales_data[test_id]

        audio_id = audio_datapoint['audio_id']
        gold_word_list = audio_datapoint['sentence']
        gold_text = " ".join(gold_word_list)
        gold_rationales = audio_datapoint['rationale']

        transcript = transcript_dict[test_id][0]['text'][1:]
        transcript_word_list = [word['word'] for word in transcript_dict[test_id][0]['words']]

        # transcript_rationales 0으로 초기화
        len_transcript_word = len(transcript_word_list)
        len_transcript_rationales = len(gold_rationales)

        transcript_rationales = [len_transcript_word * [0] for _ in range(len_transcript_rationales)]

        error = jiwer.process_words(gold_text, transcript)
        alignments = error.alignments[0]

        for alignment in alignments:
            align_type = alignment.type
            ref_start_idx = alignment.ref_start_idx
            ref_end_idx = alignment.ref_end_idx
            hyp_start_idx = alignment.hyp_start_idx
            hyp_end_idx = alignment.hyp_end_idx

            # align이 맞을 경우, ref의 rationale을 hyp에 넣는다.
            for idx, gold_rationale in enumerate(gold_rationales):
                if align_type=='equal':
                    transcript_rationales[idx][hyp_start_idx:hyp_end_idx] = gold_rationale[ref_start_idx:ref_end_idx]
                    
        transcript_rationales_dict[audio_id] = transcript_rationales


        

IndexError: list index out of range

In [6]:
file_path = '/home/jinmyeong/code/hts/classifier/Data/transcript_rationales.json'
with open(file_path, "w") as f:
    json.dump(transcript_rationales_dict, f)

In [12]:
print(transcript_rationales_dict["25502565_gab"])
transcript = transcript_dict["25502565_gab"][0]['text'][1:]
print(transcript)
transcript_word_list = [word['word'] for word in transcript_dict["25502565_gab"][0]['words']]
print(transcript_word_list)

[]
Come to an amicable agreement and resolution with the non-Kurdish Iraqi herd of instances of Iraqi Arabs being barred from returning to their homes in Kirkuk.
['Come', 'to', 'an', 'amicable', 'agreement', 'and', 'resolution', 'with', 'the', 'non-Kurdish', 'Iraqi', 'herd', 'of', 'instances', 'of', 'Iraqi', 'Arabs', 'being', 'barred', 'from', 'returning', 'to', 'their', 'homes', 'in', 'Kirkuk.']


In [13]:
audio_datapoint = audio_rationales_data["25502565_gab"]
gold_rationales = audio_datapoint['rationale']
print(gold_rationales)
print(audio_datapoint['sentence'])

[]
['come', 'to', 'an', 'amicable', 'agreement', 'and', 'resolution', 'with', 'the', 'non', 'kurdish', 'iraqi', 'heard', 'of', 'instances', 'of', 'iraqi', 'arabs', 'being', 'barred', 'from', 'returning', 'to', 'their', 'homes', 'in', 'kirkuk']
